In [ ]:
# @title Install & imports
!pip -q install pyyaml pandas unidecode

import os, re, yaml, json, unicodedata
import pandas as pd
from unidecode import unidecode

pd.set_option("display.max_colwidth", 200)


In [ ]:
# --- Upload & extract a .zip of your YAML/JSON folder (simple & robust) ---

from google.colab import files
import zipfile, io, os, shutil

# 1) Upload the ZIP (e.g., All_Conversations.zip)
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")

zip_name = next(iter(uploaded))
if not zip_name.lower().endswith(".zip"):
    raise SystemExit(f"Expected a .zip file, got: {zip_name}")

# 2) Clean extract target and unzip to /content/data
DATA_DIR = "/content/data"
if os.path.exists(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.makedirs(DATA_DIR, exist_ok=True)

with zipfile.ZipFile(io.BytesIO(uploaded[zip_name]), "r") as z:
    z.extractall(DATA_DIR)

# 3) Quick sanity: count YAML/JSON (recursively) and show a few
CALL_FILE_EXTS = (".yaml", ".yml", ".json")
found_all, found_yaml, found_json = [], [], []

for root, dirs, files in os.walk(DATA_DIR):
    # skip common noise dirs
    dirs[:] = [d for d in dirs if d not in {'.ipynb_checkpoints', '__MACOSX', '.git', '.svn'}]
    for f in files:
        fl = f.lower()
        if fl.endswith(CALL_FILE_EXTS):
            p = os.path.join(root, f)
            found_all.append(p)
            if fl.endswith((".yaml", ".yml")):
                found_yaml.append(p)
            elif fl.endswith(".json"):
                found_json.append(p)

print(f"ZIP extracted to: {DATA_DIR}")
print(f"Found {len(found_all)} transcript file(s): {len(found_json)} JSON, {len(found_yaml)} YAML")

if found_json:
    print("\nSample JSON files:")
    for p in sorted(found_json)[:10]:
        print(" -", p)

if found_yaml:
    print("\nSample YAML files:")
    for p in sorted(found_yaml)[:10]:
        print(" -", p)

if not found_all:
    print("\nNo YAML/JSON files found. Check that your ZIP contains .json/.yaml/.yml files.\n"
          "If they’re inside a nested folder, that’s fine—this search is recursive.")


In [ ]:
# @title Discovery & Loader (JSON, recursive + NFKC helper)
import os, re, json, unicodedata, pandas as pd

EXCLUDED_DIRS = {".ipynb_checkpoints", "__MACOSX", ".git", ".svn"}
JSON_EXTS = (".json",)
TIME_RX = re.compile(r"^\d{1,2}:\d{2}:\d{2}(?:\.\d+)?$")

def iter_json_files(root: str):
    """Yield JSON file paths recursively, skipping checkpoint/hidden dirs."""
    for dirpath, dirnames, filenames in os.walk(root):
        dirnames[:] = [d for d in dirnames if d not in EXCLUDED_DIRS and not d.startswith(".")]
        for fn in filenames:
            if fn.lower().endswith(JSON_EXTS) and not fn.startswith("."):
                yield os.path.join(dirpath, fn)

def _to_seconds(t):
    """Accept float/int seconds or 'HH:MM:SS(.ms)' → seconds (float)."""
    if isinstance(t, (int, float)):
        return float(t)
    if isinstance(t, str):
        t = t.strip()
        if TIME_RX.match(t):
            h, m, s = t.split(":")
            return int(h)*3600 + int(m)*60 + float(s)
        try:
            return float(t)
        except:
            return float("nan")
    return float("nan")

def _norm_speaker(spk: str) -> str:
    """Map various spellings to 'agent' or 'customer'."""
    s = (spk or "").strip().lower()
    if "agent" in s: return "agent"
    if "customer" in s or "caller" in s or "borrow" in s: return "customer"
    return "customer"

def nfkc(text: str) -> str:
    """
    NFKC-normalize for predictable processing.
    Example: 'I\\u2019m' (curly apostrophe) → "I'm" (straight apostrophe).
    """
    if text is None: return ""
    return unicodedata.normalize("NFKC", text)

def load_call_json(path, call_id=None):
    """Load one JSON transcript as a list of normalized utterance dicts."""
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Expect list of utterances; if dict, try common key 'utterances'
    items = data
    if isinstance(data, dict):
        items = data.get("utterances", [])
    if not isinstance(items, list):
        raise ValueError(f"Unrecognized JSON structure in {path}")

    utts = []
    for i, u in enumerate(items or []):
        text_raw = (u.get("text") or u.get("utterance") or u.get("content") or "")
        utts.append({
            "call_id": call_id,
            "idx": i,
            "speaker": _norm_speaker(u.get("speaker", "")),
            "text": text_raw,             # keep original (as in file)
            "text_nfkc": nfkc(text_raw),  # NFKC-normalized (for matching)
            "stime": _to_seconds(u.get("stime")),
            "etime": _to_seconds(u.get("etime")),
        })
    # Preserve overlaps; just sort by time
    utts.sort(key=lambda r: (r["stime"], r["etime"], r["idx"]))
    return utts

def load_all_calls(data_dir: str):
    """Load all JSONs, dedupe by call_id (filename stem), and report ingestion."""
    seen, rows, loaded, dupes = set(), [], [], []
    for path in sorted(iter_json_files(data_dir)):
        call_id = os.path.splitext(os.path.basename(path))[0]
        if call_id in seen:
            dupes.append(path); continue
        try:
            rows.extend(load_call_json(path, call_id=call_id))
            seen.add(call_id); loaded.append(path)
        except Exception as e:
            print(f"[WARN] Failed to parse {path}: {e}")
    return pd.DataFrame(rows), loaded, dupes

# Run
df, files_loaded, dupes = load_all_calls(DATA_DIR)
print("JSON files loaded:", len(files_loaded), "| dupes skipped:", len(dupes))
print("Distinct calls:", df['call_id'].nunique(), "| Total utterances:", len(df))
df.head(8)


In [ ]:
# Q1: Profanity detection

# Fixed frequent profanity terms in your data
PROFANITY_TERMS = [
    "fuck", "shit", "bitch", "asshole", "bastard", "damn", "crap",
    "son of a bitch", "piece of shit", "shut the fuck up",
]

# Optional: words that look similar but are benign; helps avoid silly false positives
WHITELIST = {"passion", "assess", "assignment", "scunthorpe"}

import re, os
from typing import List, Dict

def compile_q1_patterns(terms: List[str]):
    """Build two lists of regexes: single words and multi-word phrases."""
    word_patterns = []
    phrase_patterns = []
    for term in terms:
        term_low = term.lower().strip()
        if " " in term_low:
            # Phrase: just join tokens with spaces; transcripts are speech → we expect spaces
            tokens = [re.escape(t) for t in term_low.split()]
            rx = re.compile(r"(?<!\w)" + r"\s+".join(tokens) + r"(?!\w)", re.I)
            phrase_patterns.append(rx)
        else:
            # Single word with word boundaries
            rx = re.compile(rf"(?<!\w){re.escape(term_low)}(?!\w)", re.I)
            word_patterns.append(rx)
    return word_patterns, phrase_patterns

Q1_WORD_RX, Q1_PHRASE_RX = compile_q1_patterns(PROFANITY_TERMS)

def find_profanity_in_text(text_nfkc: str) -> List[Dict]:
    """Return a list of matches in one utterance (each match has type, start, end, substr)."""
    hits = []
    if not text_nfkc:
        return hits

    # single words
    for rx in Q1_WORD_RX:
        for m in rx.finditer(text_nfkc):
            snippet = text_nfkc[m.start():m.end()]
            if snippet.lower() in WHITELIST:
                continue
            hits.append({"type": "word", "start": m.start(), "end": m.end(), "substr": snippet})

    # phrases
    for rx in Q1_PHRASE_RX:
        for m in rx.finditer(text_nfkc):
            snippet = text_nfkc[m.start():m.end()]
            hits.append({"type": "phrase", "start": m.start(), "end": m.end(), "substr": snippet})

    return hits

def q1_detect_on_df(df):
    """Loop over all utterances and record where we found profanity."""
    records = []
    for row in df.itertuples(index=False):
        matches = find_profanity_in_text(row.text_nfkc)
        if matches:
            records.append({
                "call_id": row.call_id,
                "idx": row.idx,
                "speaker": row.speaker,   # 'agent' or 'customer'
                "text": row.text,         # original text (helps when reading results)
                "hits": matches
            })
    return pd.DataFrame(records)

def q1_summarize(hits_df):
    """Make one row per call: did the agent use profanity? did the customer?"""
    if hits_df.empty:
        empty_summary = pd.DataFrame(columns=["call_id","agent_used_profanity","customer_used_profanity"])
        return empty_summary, hits_df

    tmp = (hits_df.assign(flag=1)
                 .pivot_table(index="call_id", columns="speaker", values="flag", aggfunc="max", fill_value=0)
                 .rename(columns={"agent":"agent_used_profanity", "customer":"customer_used_profanity"})
                 .reset_index())

    # Ensure both columns exist even if one speaker never matched
    for c in ["agent_used_profanity","customer_used_profanity"]:
        if c not in tmp.columns:
            tmp[c] = 0

    tmp["agent_used_profanity"] = tmp["agent_used_profanity"].astype(bool)
    tmp["customer_used_profanity"] = tmp["customer_used_profanity"].astype(bool)
    return tmp, hits_df.sort_values(["call_id","idx"])

# run Q1
q1_hits = q1_detect_on_df(df)
q1_summary, q1_details = q1_summarize(q1_hits)

# make utterance numbers human-friendly (1-based) ---
if not q1_details.empty:
    q1_details = q1_details.assign(utterance_no=(q1_details["idx"] + 1).astype(int))
    # put the human-facing column up front; keep everything else
    preferred = ["call_id", "utterance_no", "speaker", "text", "hits", "idx"]
    q1_details = q1_details[[c for c in preferred if c in q1_details.columns]
                            + [c for c in q1_details.columns if c not in preferred]]

print("Q1: calls with AGENT profanity =", q1_summary.query("agent_used_profanity").shape[0])
print("Q1: calls with CUSTOMER profanity =", q1_summary.query("customer_used_profanity").shape[0])
display(q1_summary.head(8))

# ---- save Q1 outputs
os.makedirs("outputs", exist_ok=True)
q1_summary.to_csv("outputs/q1_profanity_summary_by_call.csv", index=False)
q1_details.to_csv("outputs/q1_profanity_utterance_details.csv", index=False)
print("Saved Q1 to ./outputs/")


In [ ]:
# Q2: Privacy & Compliance — final regex + state machine:

import re, os, pandas as pd
from math import inf

# Speaker normalization + sorting
BORROWER_ALIASES = {"borrower", "customer", "caller", "client", "user"}

def is_agent(spk: str) -> bool:
    return (spk or "").strip().lower() == "agent"

def is_borrower(spk: str) -> bool:
    return (spk or "").strip().lower() in BORROWER_ALIASES

def _sf(x):
    try: return float(x)
    except: return inf  # NaNs/None sort to the end

# Agent verification prompts (open "pending" window)
VERIFY_PROMPT_PATTERNS = [
    r"\b(verify|verification|confirm|confirmation)\b.*\b(date of birth|dob)\b",
    r"\b(verify|confirm)\b.*\b(address|street|city|zip|zipcode|zip\s*code|postal|postcode|pin\s*code|pincode)\b",
    r"\b(verify|confirm)\b.*\b(ssn|social security|last\s*4|last four)\b",
    r"\b(for (?:security|verification))\b.*\b(dob|date of birth|address|ssn|last\s*4|last four)\b",
]
VERIFY_PROMPT_RX = [re.compile(p, re.I) for p in VERIFY_PROMPT_PATTERNS]

# Sensitive disclosures (must NOT occur before verified=True)
SENSITIVE_PATTERNS = [
    r"\bcurrent\s+balance\b(?:[^0-9]*|\s*is\s*)\$?\d[\d,]*(?:\.\d{2})?",
    r"\bbalance\b(?:[^0-9]*|\s*is\s*)\$?\d[\d,]*(?:\.\d{2})?",
    r"\bamount(?:\s*due)?\b(?:[^0-9]*|\s*is\s*)\$?\d[\d,]*(?:\.\d{2})?",
    r"\byou\s+owe\b.*?\$?\d[\d,]*(?:\.\d{2})?",
    r"\bpayment\s+(?:posted|received)\b.*?\$?\d[\d,]*(?:\.\d{2})?",
    r"\bwe\s+received(?:\s+your)?\s+payment\b.*?\$?\d[\d,]*(?:\.\d{2})?",
    r"\b(account|acct|a\/c)\s*(?:no\.?|number|#)?\s*[:#]?\s*\d{3,}\b",
    r"\b(?:ending\s+in|last\s*(?:4|four))\b.*?\b\d{4}\b",
    r"\bssn\b.*?\b\d{3}[-\s]?\d{2}[-\s]?\d{4}\b",
]
SENSITIVE_RX = [re.compile(p, re.I) for p in SENSITIVE_PATTERNS]

# Verification evidence from borrower

# Numeric DOBs
DOB_NUMERIC_RXES = [
    r"\b(0?[1-9]|1[0-2])[\/\-\.](0?[1-9]|[12]\d|3[01])[\/\-\.](\d{2,4})\b",  # MM/DD/YYYY or MM-DD-YY
    r"\b(0?[1-9]|[12]\d|3[01])[\/\-\.](0?[1-9]|1[0-2])[\/\-\.](\d{2,4})\b",  # DD/MM/YYYY or DD-MM-YY
    r"\b(\d{4})[\/\-\.](0?[1-9]|1[0-2])[\/\-\.](0?[1-9]|[12]\d|3[01])\b",    # YYYY-MM-DD or YYYY.MM.DD
]
DOB_NUMERIC_RXES = [re.compile(p, re.I) for p in DOB_NUMERIC_RXES]

# Month-name DOBs (with/without ordinals, comma, "of")
MONTH = r"(jan|feb|mar|apr|may|jun|jul|aug|sep|sept|oct|nov|dec|january|february|march|april|june|july|august|september|october|november|december)"
DAY   = r"(0?[1-9]|[12]\d|3[01])(?:st|nd|rd|th)?"
YEAR  = r"\d{2,4}"
DOB_TEXT_RXES = [
    rf"\b{MONTH}\s+{DAY}\s*,?\s*{YEAR}\b",           # January 15, 1990 / Jan 15 90
    rf"\b{DAY}\s+(?:of\s+)?{MONTH}\s*,?\s*{YEAR}\b", # 15th of January 1990 / 15 Jan 90
    rf"\b{YEAR}\s+{MONTH}\s+{DAY}\b",                # 1990 January 15
]
DOB_TEXT_RXES = [re.compile(p, re.I) for p in DOB_TEXT_RXES]

# Contiguous 8-digit dates (use only while pending): YYYYMMDD or DDMMYYYY
CONTIG_8_RX = re.compile(r"\b(\d{8})\b")
def _looks_like_contig_date(s: str) -> bool:
    m = CONTIG_8_RX.search(s)
    if not m: return False
    d = m.group(1)
    try:  # try YYYYMMDD
        yyyy, mm, dd = int(d[0:4]), int(d[4:6]), int(d[6:8])
        if 1900 <= yyyy <= 2025 and 1 <= mm <= 12 and 1 <= dd <= 31: return True
    except: pass
    try:  # try DDMMYYYY
        dd, mm, yyyy = int(d[0:2]), int(d[2:4]), int(d[4:8])
        if 1900 <= yyyy <= 2025 and 1 <= mm <= 12 and 1 <= dd <= 31: return True
    except: pass
    return False

# SSN / last-4 and guards
SSN_FULL_RX    = re.compile(r"\b\d{3}[-\s]?\d{2}[-\s]?\d{4}\b")
NINE_DIGIT_RX  = re.compile(r"\b\d{9}\b")
FOUR_DIGIT_RX  = re.compile(r"\b\d{4}\b")
MONEY_HINT_RX  = re.compile(r"\$|\b(usd|dollars?)\b|\d+,\d{3}", re.I)
PHONE_HINT_RX  = re.compile(r"\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4})\b")
TOO_LONG_RX    = re.compile(r"\d{10,}")  # >9 contiguous digits

# Address recognition
STREET_LINE_RX = re.compile(
    r"""
    \b
    \d{1,6}[A-Za-z]?                           # house number (e.g., 12 or 12A)
    (?:\s+(?:N|S|E|W|NE|NW|SE|SW)\b\.?)?       # optional direction
    \s+[A-Za-z0-9][A-Za-z0-9'’\-\.]*           # first street token (Elm, O'Neil)
    (?:\s+[A-Za-z0-9'’\-\.]+){0,4}             # optional extra name tokens
    \s+
    (?:st|street|rd|road|ave|avenue|blvd|boulevard|dr|drive|ln|lane|
       ct|court|pl|place|ter|terrace|cir|circle|way|pkwy|parkway|hwy|highway)\.?
    \b
    """, re.I | re.VERBOSE
)

ADDRESS_HINT_RX = re.compile(
    r"\b("
    r"street|st\.?|ave|avenue|blvd|boulevard|road|rd\.?|drive|dr\.?|lane|ln\.?|terrace|ter\.?|court|ct\.?|place|pl\.?|"
    r"way|circle|cir\.?|parkway|pkwy\.?|highway|hwy\.?|"
    r"apt|apartment|unit|suite|ste\.?|bldg|building|fl|floor|#\s*\w+|"
    r"po\s*box|p\.?o\.?\s*box|box\s*\d+|"
    r"\d{5}(?:-\d{4})?|zip|zipcode|zip\s*code|postal|postcode|pin\s*code|pincode|"
    r"city|state"
    r")\b", re.I
)

def borrower_provided_verification(txt: str, *, pending_only: bool = True) -> bool:
    t = (txt or "").strip()

    # 1) DOB (numeric / month-name / contiguous 8-digit while pending)
    if any(rx.search(t) for rx in DOB_NUMERIC_RXES): return True
    if any(rx.search(t) for rx in DOB_TEXT_RXES):    return True
    if pending_only and _looks_like_contig_date(t):  return True

    # 2) SSN (formatted) or bare 9 digits (not money/phone/too-long)
    if SSN_FULL_RX.search(t): return True
    if pending_only and NINE_DIGIT_RX.search(t):
        if not (MONEY_HINT_RX.search(t) or PHONE_HINT_RX.search(t) or TOO_LONG_RX.search(t)):
            return True

    # 3) Last-4 as bare 4 digits (answer-like, only while pending)
    if pending_only and (FOUR_DIGIT_RX.fullmatch(t) or re.fullmatch(r"\d{4}\D{0,2}", t)):
        return True

    # 4) Address: strong street line first; then general hints
    if STREET_LINE_RX.search(t): return True
    if ADDRESS_HINT_RX.search(t): return True

    return False

# Q2 core checker

def q2_check_one_call(utterances):
    """
    Input: list of utterance dicts with keys:
           speaker, text, text_nfkc, stime, etime, idx
    Output: dict { non_compliant: bool, verified_observed: bool, violations: [..] }
    """
    verified = False
    pending_verify = False
    violations = []

    # robust ordering
    utts = sorted(utterances, key=lambda r: (_sf(r.get("stime")), _sf(r.get("etime")), r.get("idx", 0)))

    for u in utts:
        spk = (u.get("speaker") or "").lower()
        txt = (u.get("text_nfkc") or u.get("text") or "")

        # 1) Agent prompts → open verification window
        if is_agent(spk) and any(rx.search(txt) for rx in VERIFY_PROMPT_RX):
            pending_verify = True
            continue

        # 2) Borrower provides verification during pending window
        if is_borrower(spk) and pending_verify:
            if borrower_provided_verification(txt, pending_only=True):
                verified = True
                pending_verify = False
                continue

        # 3) Sensitive disclosure before verified → violation
        if is_agent(spk) and any(rx.search(txt) for rx in SENSITIVE_RX):
            if not verified:
                violations.append({
                    "idx": u.get("idx", -1),
                    "text": u.get("text", txt),
                    "reason": "sensitive_before_verification"
                })

    return {
        "non_compliant": bool(violations),
        "verified_observed": verified,
        "violations": violations
    }

def q2_detect_on_df(df: pd.DataFrame):
    results, details = [], []
    for call_id, g in df.groupby("call_id", sort=True):
        utts = g.sort_values(["stime","etime","idx"]).to_dict("records")
        res = q2_check_one_call(utts)
        results.append({
            "call_id": call_id,
            "non_compliant": res["non_compliant"],
            "verified_observed": res["verified_observed"],
        })
        for v in res["violations"]:
            details.append({"call_id": call_id, **v})
    return pd.DataFrame(results), pd.DataFrame(details)

# Run + Save
q2_summary, q2_details = q2_detect_on_df(df)

if not q2_details.empty:
    q2_details = q2_details.assign(utterance_no=(q2_details["idx"] + 1).astype(int))
    preferred = ["call_id", "utterance_no", "reason", "text", "idx"]
    q2_details = q2_details[[c for c in preferred if c in q2_details.columns]
                            + [c for c in q2_details.columns if c not in preferred]]

os.makedirs("outputs", exist_ok=True)
q2_summary.to_csv("outputs/q2_privacy_noncompliance_by_call.csv", index=False)
q2_details.to_csv("outputs/q2_privacy_violation_details.csv", index=False)

print("Q2: non-compliant calls =", int(q2_summary.query("non_compliant").shape[0]))

In [ ]:
# === Final merge → single CSV for Streamlit ===
# Produces: outputs/final_calls_review.csv

import os, glob, json, re
import pandas as pd

# --- CONFIG ---
DATA_DIR = "./data"                 # folder with original JSON calls (set to None if unavailable)
OUT_DIR  = "./outputs"
FINAL_CSV = os.path.join(OUT_DIR, "regex_summary.csv")

Q1_SUMMARY = os.path.join(OUT_DIR, "q1_profanity_summary_by_call.csv")
Q1_DETAILS = os.path.join(OUT_DIR, "q1_profanity_utterance_details.csv")
Q2_SUMMARY = os.path.join(OUT_DIR, "q2_privacy_noncompliance_by_call.csv")
Q2_DETAILS = os.path.join(OUT_DIR, "q2_privacy_violation_details.csv")

os.makedirs(OUT_DIR, exist_ok=True)

def call_id_from_path(p: str) -> str:
    b = os.path.basename(p)
    return os.path.splitext(b)[0]

# --- 1) Get the authoritative list of call_ids (aim for the full 249) ---
master_call_ids = []

if DATA_DIR and os.path.isdir(DATA_DIR):
    json_paths = []
    for pat in ("**/*.json", "*.json"):
        json_paths.extend(glob.glob(os.path.join(DATA_DIR, pat), recursive=True))
    master_call_ids = sorted({call_id_from_path(p) for p in json_paths})

# If we don't have DATA_DIR or it’s empty, fall back to union from existing CSVs
dfs_present = []
for p in (Q1_SUMMARY, Q1_DETAILS, Q2_SUMMARY, Q2_DETAILS):
    if os.path.isfile(p):
        dfs_present.append(pd.read_csv(p))

if not master_call_ids and dfs_present:
    ids = set()
    for d in dfs_present:
        # Handle either `call_id` present or nested columns
        if "call_id" in d.columns:
            ids.update(d["call_id"].dropna().astype(str))
    master_call_ids = sorted(ids)

# Wrap in a starter DataFrame
base = pd.DataFrame({"call_id": master_call_ids}).astype({"call_id": "string"})

# --- 2) Load Q1 (profanity) summary & details ---
q1_sum = pd.read_csv(Q1_SUMMARY) if os.path.isfile(Q1_SUMMARY) else pd.DataFrame(columns=["call_id","agent_used_profanity","customer_used_profanity"])
q1_sum = q1_sum.rename(columns={
    "agent_used_profanity": "profanity_agent",
    "customer_used_profanity": "profanity_customer",
})
for col in ("profanity_agent","profanity_customer"):
    if col not in q1_sum.columns: q1_sum[col] = False
q1_sum["call_id"] = q1_sum.get("call_id","").astype("string")

# Utterance numbers that had profanity (from details)
q1_det = pd.read_csv(Q1_DETAILS) if os.path.isfile(Q1_DETAILS) else pd.DataFrame(columns=["call_id","utterance_no"])
if not q1_det.empty:
    # Collect all profanity utterance_no per call, sorted unique, as comma-separated string
    hits_by_call = (
        q1_det.dropna(subset=["call_id","utterance_no"])
             .assign(utterance_no=lambda d: d["utterance_no"].astype(int))
             .sort_values(["call_id","utterance_no"])
             .groupby("call_id")["utterance_no"]
             .apply(lambda s: ",".join(map(str, sorted(set(s.tolist())))))
             .reset_index()
             .rename(columns={"utterance_no": "profanity_utterance"})
    )
else:
    hits_by_call = pd.DataFrame(columns=["call_id","profanity_utterance"])
hits_by_call["call_id"] = hits_by_call.get("call_id","").astype("string")

# --- 3) Load Q2 (privacy/compliance) summary & details ---
q2_sum = pd.read_csv(Q2_SUMMARY) if os.path.isfile(Q2_SUMMARY) else pd.DataFrame(columns=["call_id","non_compliant","verified_observed"])
q2_sum["call_id"] = q2_sum.get("call_id","").astype("string")
if "verified_observed" not in q2_sum.columns:
    q2_sum["verified_observed"] = False
if "non_compliant" not in q2_sum.columns:
    q2_sum["non_compliant"] = False

q2_det = pd.read_csv(Q2_DETAILS) if os.path.isfile(Q2_DETAILS) else pd.DataFrame(columns=["call_id","reason"])
q2_det["call_id"] = q2_det.get("call_id","").astype("string")

# Info shared before verification → look for reason tokens that indicate pre-verification disclosure
PREVERIF_TOKENS = (
    "sensitive_before_verification",
    "before_verification",
    "disclosure_before_verification",
)
if not q2_det.empty and "reason" in q2_det.columns:
    preverif = (
        q2_det.assign(reason=q2_det["reason"].astype(str).str.lower())
              .assign(flag=lambda d: d["reason"].apply(lambda r: any(tok in r for tok in PREVERIF_TOKENS)))
              .groupby("call_id")["flag"].any()
              .reset_index()
              .rename(columns={"flag": "info_shared_without_identity_verification"})
    )
else:
    preverif = pd.DataFrame(columns=["call_id","info_shared_without_identity_verification"])
preverif["call_id"] = preverif.get("call_id","").astype("string")
if "info_shared_without_identity_verification" not in preverif.columns:
    preverif["info_shared_without_identity_verification"] = False

# --- 4) Merge everything LEFT onto the full call list ---
final = base.merge(q1_sum[["call_id","profanity_agent","profanity_customer"]], on="call_id", how="left")
final = final.merge(hits_by_call[["call_id","profanity_utterance"]], on="call_id", how="left")
final = final.merge(q2_sum[["call_id","verified_observed","non_compliant"]], on="call_id", how="left")
final = final.merge(preverif[["call_id","info_shared_without_identity_verification"]], on="call_id", how="left")

# --- 5) Derive the three Q2 flags you requested ---
final["verified_identity"] = final["verified_observed"].fillna(False)

# If any pre-verification disclosure detected → True
final["info_shared_without_identity_verification"] = final["info_shared_without_identity_verification"].fillna(False)

# "no_info_or_identity_shared":
# True when:
#   - no verification observed, AND
#   - no pre-verification info shared, AND
#   - not otherwise marked non_compliant
final["no_info_or_identity_shared"] = (
    (~final["verified_identity"]) &
    (~final["info_shared_without_identity_verification"]) &
    (~final["non_compliant"].fillna(False))
)

# --- 6) Clean up booleans & blanks ---
for col in ["profanity_agent","profanity_customer","verified_identity",
            "info_shared_without_identity_verification","no_info_or_identity_shared"]:
    if col not in final.columns:
        final[col] = False
    final[col] = final[col].fillna(False).astype(bool)

final["profanity_utterance"] = final["profanity_utterance"].fillna("").astype(str)

# Optional: drop helper columns
final = final.drop(columns=["verified_observed","non_compliant"], errors="ignore")

# --- 7) Save ---
final = final.sort_values("call_id")
final.to_csv(FINAL_CSV, index=False)

print(f"Saved: {FINAL_CSV}")
print(f"Rows (calls): {len(final)}")
print(final.head(10))
